## RNA-seq data analysis using python

#### Bowtie2 Module:
Bowtie 2 is used for aligning RNA-seq data (Fastq) to reference sequences. Bowtie2 can be downloaded via the link https://sourceforge.net/projects/bowtie-bio/files/bowtie2/2.4.5/. For this demonstration, bowtie2-2.4.5-mingw-x86_64 was downloaded and employed in RNA-seq data analysis.

#### RNA-seq sample data sets
For this demonstration, 12 RNA-seq data sets (Fastq files) were downloaded from Zenodo https://zenodo.org/record/4249555#.YvhsYmnMKpo. They are small portions of original Fastq files for program development. Their metadata are follows:

    FileName	                      SampleName	CellType	Status	SRAFileName
    MCL1.DG_BC2CTUACXX_ACTTGA_L002_R1	MCL1.DG	basal	virgin	SRR1552450.fastq.gz
    MCL1.DH_BC2CTUACXX_CAGATC_L002_R1	MCL1.DH	basal	virgin	SRR1552451.fastq.gz
    MCL1.DI_BC2CTUACXX_ACAGTG_L002_R1	MCL1.DI	basal	pregnant	SRR1552452.fastq.gz
    MCL1.DJ_BC2CTUACXX_CGATGT_L002_R1	MCL1.DJ	basal	pregnant	SRR1552453.fastq.gz
    MCL1.DK_BC2CTUACXX_TTAGGC_L002_R1	MCL1.DK	basal	lactate	SRR1552454.fastq.gz
    MCL1.DL_BC2CTUACXX_ATCACG_L002_R1	MCL1.DL	basal	lactate	SRR1552455.fastq.gz
    MCL1.LA_BC2CTUACXX_GATCAG_L001_R1	MCL1.LA	luminal	virgin	SRR1552444.fastq.gz
    MCL1.LB_BC2CTUACXX_TGACCA_L001_R1	MCL1.LB	luminal	virgin	SRR1552445.fastq.gz
    MCL1.LC_BC2CTUACXX_GCCAAT_L001_R1	MCL1.LC	luminal	pregnant	SRR1552446.fastq.gz
    MCL1.LD_BC2CTUACXX_GGCTAC_L001_R1	MCL1.LD	luminal	pregnant	SRR1552447.fastq.gz
    MCL1.LE_BC2CTUACXX_TAGCTT_L001_R1	MCL1.LE	luminal	lactate	SRR1552448.fastq.gz
    MCL1.LF_BC2CTUACXX_CTTGTA_L001_R1	MCL1.LF	luminal	lactate	SRR1552449.fastq.gz

#### Mouse CDS reference sequences
For this demonstration, mouse CDS (protein coding DNA sequence) sequences (Mus_musculus.GRCm39.cds.all.fa) were downloaded via http://ftp.ensembl.org/pub/release-107/fasta/mus_musculus/cds/ and used as reference sequences for alignment analysis by bowtie2 due to the smaller size of its file. For comprehensive RNA-seq data analysis, mouse cDNA sequences are the better choice, which can be downloaded via http://ftp.ensembl.org/pub/release-107/fasta/mus_musculus/cdna/.

#### Application files in the bowtie2 module
There are 9 total binary files in downloaded bowtie2-2.4.5-mingw-x86_64:
* bowtie2-build - wrapper script for bowtie2-build-l and bowtie2-build-s
* bowtie2-build-l - actual script called by bowtie2-build for building large reference index, result file will have .bt2l postfix
* bowtie2-build-s - actual script called by bowtie2-build for building small reference index, result file will have .bt2 postfix
* bowtie2-inspect - wrapper script for bowtie2-inspect-l and bowtie2-inspect-s
* bowtie2-inspect-l - actual script called by bowtie2-inspect for long bowtie2 index files
* bowtie2-inspect-s - actual script called by bowtie2-inspect for short bowtie2 index files
* bowtie2 - wrapper script for bowtie2-align-l and bowtie2-align-s
* bowtie2-align-l - actual script called by bowtie2 for long read alignment
* bowtie2-align-s - actual script called by bowtie2 for short read alignment

To run bowtie2 wrapper (a perl script), a Unix OS system environment is needed. As this demonstration was under the windows OS environment, their specific applications (.exe) need to be executed for mapping of sequencing reads to reference sequences. Based on the length of sequencing reads and size of reference sequences, respective applications are used (-l for large and -s for small).

In [1]:
# Use bowtie2-build-l to index mouse CDS sequences
import subprocess
subprocess.call("bowtie2-build-l.exe ./Ms_RNA-seq/reference/Mus_musculus.GRCm39.cds.all.fa ./Ms_RNA-seq/reference/mouse_genes")

0

In [2]:
# Use the os module to list all fastq file names and assign file names to an variable (seq_IDs)
import os

files = os.listdir("./Ms_RNA-seq/seq_reads/")

seq_IDs = []
for file in files:
    if "SRR" in file:
        seq_IDs.append(file.replace(".fastq", ""))

print(len(seq_IDs))
print(seq_IDs)

12
['SRR1552444', 'SRR1552445', 'SRR1552446', 'SRR1552447', 'SRR1552448', 'SRR1552449', 'SRR1552450', 'SRR1552451', 'SRR1552452', 'SRR1552453', 'SRR1552454', 'SRR1552455']


In [3]:
# Use a for loop to automatically aligning fastq files to indexed mouse CDS reference sequences by bowtie-align-l.exe
for seq_ID in seq_IDs:    
    subprocess.call(f"bowtie2-align-l.exe -x ./Ms_RNA-seq/reference/mouse_genes -U ./Ms_RNA-seq/seq_reads/{seq_ID}.fastq. -S ./Ms_RNA-seq/seq_sam/{seq_ID}.sam")

In [4]:
# Iterate each sam file generated by bowtie2 and use regular expression to count the total number of a transcript/CDS ID 
# appearing in the individual sam file. Store transcript/CDS IDs as keys and their respective expression counts as values 
# in a dictionary for each sam file. All generated dictionaries are included in a list variable.
import re

Dict_list = []

for i in range(len(seq_IDs)):
    Dict = {}
    SAMFile = open(f"./Ms_RNA-seq/seq_sam/{seq_IDs[i]}.sam")
    
    for line in SAMFile:
        m = re.search(r"\s(ENSMUST\d*\.\d*)\s", line)

        if m:
            gene_id = m.group(1)

            if gene_id in Dict:
                Dict[gene_id] += 1
            else:
                Dict[gene_id] = 1
    Dict_list.append(Dict)  

    SAMFile.close()

In [5]:
# Extract all transcript/CDS IDs from expression count dictionaries and store them in the list variable
genes_acc_list = []

for Dict in Dict_list:
    genes_acc_list = genes_acc_list + list(Dict.keys())

# Use set() to remove the duplicate transcript/CDS IDs and generate a list of unique transcript/CDS IDs
genes_ids = set(genes_acc_list)

# Convert the set datatype into the list datatype
genes_ids = list(genes_ids)

# Sort the unique transcript/CDS IDs
genes_ids.sort()
print(genes_ids[:20])

['ENSMUST00000000080.8', 'ENSMUST00000000090.8', 'ENSMUST00000000122.7', 'ENSMUST00000000137.8', 'ENSMUST00000000312.12', 'ENSMUST00000000384.8', 'ENSMUST00000000466.13', 'ENSMUST00000000590.16', 'ENSMUST00000000608.8', 'ENSMUST00000000642.11', 'ENSMUST00000000756.6', 'ENSMUST00000000804.7', 'ENSMUST00000000809.3', 'ENSMUST00000000939.15', 'ENSMUST00000001009.14', 'ENSMUST00000001042.10', 'ENSMUST00000001043.14', 'ENSMUST00000001092.15', 'ENSMUST00000001147.5', 'ENSMUST00000001166.14']


In [6]:
# Generate a complete dictionary to store transcript/CDS IDs as keys and their respectiv gene symbols as values
# from the fasta file of mouse CDS sequences
file = open("./Ms_RNA-seq/reference/Mus_musculus.GRCm39.cds.all.fa")

genes_dict = {}

for line in file:
    if ">" in line:
        items = line.strip().split()
        for item in items:
            if ">" in item:
                id = item.replace(">", "")
                genes_dict[id] = ""
            elif "gene_symbol" in item:
                genes_dict[id] = item.replace("gene_symbol:", "")
file.close()


In [7]:
# Generate a dictionary storing transcript/CDS IDs extracted from expression count dictionaries as keys 
# and gene symbols extracted from the complete gene-ID-symbol dictionary as values
genes_id_symbol_dict = {}

for id in genes_ids:
    genes_id_symbol_dict[id] = genes_dict[id]

In [8]:
# Include all transcript/CDS IDs in expression count dictionaries and assign 0 count to those transcript/CDS IDs 
# are not present in individual expression count dictionaries
for i in range(len(Dict_list)):
    for id in genes_ids:
        if id in Dict_list[i]:
            continue
        else:
            Dict_list[i][id] = 0

In [9]:
# Re-organize expression count dictionaries according to the order of transcript/CDS IDs in genes_ids
Count_Dict  = {}
Count_Dict_list = []

for i in range(len(Dict_list)):
    Count_Dict  = {}
    
    for id in genes_ids:
        Count_Dict[id] = Dict_list[i].get(id)
        
    Count_Dict_list.append(Count_Dict)

In [10]:
import pandas as pd

# Create a list variable to store multiple arrays including transcript/CDS IDs, gene symbols, 
# and expression count arrays generated from 12 sequencing fastq files
exp_count_list = []

for Count_Dict in Count_Dict_list:
    exp_count_list.append(list(Count_Dict.values()))
    
exp_count_list = [list(genes_id_symbol_dict.keys()), list(genes_id_symbol_dict.values())] + exp_count_list


# Create a dictionary storing Transcript_ID, Gene_Symbol, and 12 Sequencing IDs as keys and 
# their respective list arrays and expression count arrays as values
exp_count_dict = {}

columns = ['Transcript_ID', 'Gene_Symbol'] + seq_IDs

for n in range(len(columns)):
    exp_count_dict[columns[n]] = exp_count_list[n]

# Use pd.DataFrame to convert a dictiionary into a Data Frame data structure
# This data frame contains the expression count matrix for individual transcript/CDS IDs
exp_count_df = pd.DataFrame(exp_count_dict)

exp_count_df.head(20)

,Transcript_ID,Gene_Symbol,SRR1552444,SRR1552445,SRR1552446,SRR1552447,SRR1552448,SRR1552449,SRR1552450,SRR1552451,SRR1552452,SRR1552453,SRR1552454,SRR1552455
0,ENSMUST00000000080.8,Klf6,0,1,0,0,0,0,1,0,0,0,0,0
1,ENSMUST00000000090.8,Cox5a,0,0,0,0,0,0,1,0,0,0,0,0
2,ENSMUST00000000122.7,Ngfr,0,0,0,0,0,0,1,0,0,0,0,0
3,ENSMUST00000000137.8,Actr2,0,0,1,0,0,0,0,0,0,0,0,0
4,ENSMUST00000000312.12,Cdh1,0,0,1,0,0,0,0,0,0,0,0,0
5,ENSMUST00000000384.8,Trappc10,0,0,0,0,0,0,0,0,0,1,0,0
6,ENSMUST00000000466.13,Plin2,0,0,0,1,0,0,0,0,0,0,0,0
7,ENSMUST00000000590.16,Rdx,1,0,0,0,0,0,0,0,0,0,0,0
8,ENSMUST00000000608.8,Gm2a,0,0,0,0,0,0,0,0,0,1,0,0
9,ENSMUST00000000642.11,Hk2,0,0,0,0,0,0,0,0,0,0,0,1


In [11]:
# Export the pandas Data Frame as a CSV file
exp_count_df.to_csv("Sample_exp_count_data.csv", index=False)

In [12]:
# Extract gene symbols and use set() to remove duplicate gene symbols
gene_symbols = list(set(genes_id_symbol_dict.values()))
gene_symbols.sort()

# Generate a dictionary to store gene symbols as keys and their respective transcript/CDS IDs as values
# transcript/CDS IDs are stored in list arrays for individual gene symbols as some gene symbols 
# have multiple transcript/CDS IDs
gene_symbol_ID_dict = {}

for gene in gene_symbols:
    for id in genes_id_symbol_dict.keys():
        if genes_id_symbol_dict[id] == gene:
            if gene not in gene_symbol_ID_dict:
                gene_symbol_ID_dict[gene] = [id]
            else:
                gene_symbol_ID_dict[gene].append(id)
        else:
            continue

In [13]:
# Create a list variable to store multiple dictionaries with gene symbols as keys and 
# their respective expression counts as values. When a gene symbol has multiple transcript/CDS IDs,
# their expression counts are summed together and assigned to the gene symbol.
gene_exp_count_list = []

for Count_Dict in Count_Dict_list:
    gene_exp_count_dict = {}
    
    for gene in gene_symbol_ID_dict.keys():
        ids = gene_symbol_ID_dict.get(gene)
        gene_exp_count_dict[gene] = 0
        
        for id in ids:
            gene_exp_count_dict[gene] += Count_Dict.get(id)
            
    gene_exp_count_list.append(gene_exp_count_dict)

In [14]:
import pandas as pd

# Create a list variable to store multiple arrays including gene symbols, and expression count arrays 
# generated from 12 sequencing fastq files
exp_count_list2 = []

for exp_count in gene_exp_count_list:
    exp_count_list2.append(list(exp_count.values()))
    
exp_count_list2 = [gene_symbols] + exp_count_list2


# Create a dictionary storing Gene_Symbol, and 12 Sequencing IDs as keys and their respective list arrays 
# and expression count arrays as values
exp_count_dict2 = {}

columns = ['Gene_Symbol'] + seq_IDs

for n in range(len(columns)):
    exp_count_dict2[columns[n]] = exp_count_list2[n]


# Use pd.DataFrame to convert a dictiionary into a Data Frame data structure
# This data frame contains the expression count matrix for individual gene symbols
exp_count_df2 = pd.DataFrame(exp_count_dict2)

exp_count_df2.head(20)

,Gene_Symbol,SRR1552444,SRR1552445,SRR1552446,SRR1552447,SRR1552448,SRR1552449,SRR1552450,SRR1552451,SRR1552452,SRR1552453,SRR1552454,SRR1552455
0,,1,0,0,0,0,0,0,0,0,0,0,0
1,0610040J01Rik,1,0,0,0,0,0,0,0,0,0,0,0
2,1600014C10Rik,0,0,0,0,0,0,0,0,0,1,0,0
3,1700012B07Rik,0,0,0,0,0,1,0,0,0,0,0,0
4,1810013L24Rik,0,0,0,0,0,0,0,1,0,0,0,0
5,2010315B03Rik,1,0,0,0,0,0,0,0,0,0,0,0
6,2410002F23Rik,0,1,0,0,0,0,0,0,0,0,0,0
7,2610507B11Rik,0,0,0,0,0,0,1,1,0,0,0,1
8,2700049A03Rik,0,0,1,0,0,0,0,0,0,0,0,0
9,4932438A13Rik,0,0,0,0,0,0,0,0,0,1,0,0


In [15]:
# Export the pandas Data Frame as a CSV file
exp_count_df2.to_csv("Sample_gene_exp_count_data.csv", index=False)